Ver COMO chota se deriva lo del laplaciano a mano jeje

El laplaciano esta dado por:

$$
\Delta c = \frac{\partial^2 c}{\partial x^2} + \frac{\partial^2 c}{\partial y^2}
$$

Usando diferencias finitas
$$
\frac{\partial^2 c}{\partial x^2} \approx \frac{c(x+\Delta x, y) - 2c(x,y) + c(x-\Delta x, y)}{(\Delta x)^2}

$$

$$
\frac{\partial^2 c}{\partial y^2} \approx \frac{c(x, y+\Delta y) - 2c(x,y) + c(x, y-\Delta y)}{(\Delta y)^2}

$$

$$
\frac{d^2 f}{dx^2}(x) \approx \frac{f(x+\Delta x) - 2f(x) + f(x-\Delta x)}{(\Delta x)^2}


$$

In [ ]:
def compute_residual(c_flat, X, Y, u, v, dt, D):
    """
    Calcula el residuo F(c) = evolución(c) - c, para el sistema de advección-difusión.

    Parámetros:
    - c_flat: ndarray de forma (n²,), concentración actual aplanada en un vector.
    - X, Y: mallas espaciales 2D con las coordenadas (x, y).
    - u, v: componentes del campo de velocidad en cada punto.
    - dt: paso de tiempo.
    - D: coeficiente de difusión.

    Retorna:
    - F(c): ndarray de forma (n²,), residuo de la evolución (vector).
    """

    n = X.shape[0]
    c = c_flat.reshape((n, n))
    
    # Mover el flujo, hacia atras en el tiempo.
    # Hacia adelaste es rpegunta a donde va
    # hacia atras es pregunta de donde viene
    # Hacia atras es el enfoque lagrangeano que es mas estable.
    X_new = X - u * dt
    Y_new = Y - v * dt
    
    # Mantener las coordenadas dentro del dominio [0, 1]
    X_new = np.mod(X_new, 1)
    Y_new = np.mod(Y_new, 1)

    # ponemos las nuevas coordenadas dentro de la grilla
    coords = np.array([Y_new.flatten() * (n-1), X_new.flatten() * (n-1)])
    c_advected = map_coordinates(c, coords, order=1, mode='wrap').reshape(n, n)
    
    laplacian = (
        np.roll(c_advected, 1, axis=0) +  # abajo (vecino arriba en array)
        np.roll(c_advected, -1, axis=0) + # arriba (vecino abajo en array)
        np.roll(c_advected, 1, axis=1) +  # derecha (vecino izquierda en array)
        np.roll(c_advected, -1, axis=1)   # izquierda (vecino derecha en array)
        - 4 * c_advected                  # menos 4 veces el centro
    )
    c_diffused = c_advected + D * laplacian
    
    return (c_diffused - c).flatten()

El jacobiano tiene que tener las derivadas
$$
\frac{\partial F_i}{\partial c_j}
$$

In [ ]:
def approximate_jacobian(F, c, epsilon=1e-6):
    """
    Aproxima la matriz Jacobiana J_F de la función F en el punto c usando diferencias finitas.

    Parámetros:
    - F: función que evalúa el residuo F(c).
    - c: ndarray de forma (n²,), concentración actual aplanada.
    - epsilon: perturbación pequeña usada para diferencias finitas (default 1e-6).

    Retorna:
    - J: ndarray de forma (n², n²), matriz Jacobiana aproximada.
    """
    n2 = c.size
    J = np.zeros((n2, n2))
    F0 = F(c)  
    
    for i in range(n2):
        e_i = np.zeros(n2)
        e_i[i] = 1.0
        F_perturbed = F(c + epsilon * e_i)
        J[:, i] = (F_perturbed - F0)/epsilon
    
    return J


In [ ]:
def newton_solver(F, c0_flat, tol=1e-6, max_iter=20):
    """
    Resuelve F(c) = 0 utilizando el método de Newton clásico.

    Parámetros:
    - F: función que evalúa el residuo F(c).
    - c0_flat: ndarray de forma (n²,), concentración inicial aplanada.
    - tol: tolerancia para la norma del residuo que indica convergencia (default 1e-6).
    - max_iter: número máximo de iteraciones permitidas (default 20).

    Retorna:
    - c_steady: ndarray de forma (n²,), solución aproximada donde F(c) ≈ 0.
    """
    
    pass